# Precificação dinâmica 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Modelo: baseline

Este modelo sugere os preços dos produtos baeado na médias das categorias. 

Criado como um referencial de base. Modelos que utilizem técnicas mais avançadas devem ter desempenho superior a este modelo para serem considerados como alternativas viáveis.

Importação das bibliotecas e pacotes necessários

In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics

Leitura das bases de treino e teste

In [4]:
dados_treino = pd.read_csv('/content/drive/MyDrive/Blue Edtech/data/processed/dados_treino.csv')
dados_teste = pd.read_csv('/content/drive/MyDrive/Blue Edtech/data/processed/dados_teste.csv')
dados_validacao = pd.read_csv('/content/drive/MyDrive/Blue Edtech/data/processed/dados_validacao.csv')

Agregação das colunas de categorias em apenas uma.

In [5]:
dados_treino['categorias'] = dados_treino[['category_1', 'category_2', 'category_3']].agg('_'.join, axis=1)
dados_teste['categorias'] = dados_teste[['category_1', 'category_2', 'category_3']].agg('_'.join, axis=1)
dados_validacao['categorias'] = dados_validacao[['category_1', 'category_2', 'category_3']].agg('_'.join, axis=1)

Cálculo dos preços médios por categorias.

In [6]:
medias_categorias = dados_treino.groupby('categorias').price.mean()
medias_categorias

categorias
Beauty_Bath & Body_Bath                          18.682301
Beauty_Bath & Body_Bathing Accessories           22.400922
Beauty_Bath & Body_Cleansers                     13.502174
Beauty_Bath & Body_Other                         21.723404
Beauty_Bath & Body_Scrubs & Body Treatments      25.616438
                                                   ...    
Women_Women's Handbags_Messenger & Crossbody     58.806838
Women_Women's Handbags_Other                     32.984701
Women_Women's Handbags_Satchel                  113.467377
Women_Women's Handbags_Shoulder Bag              65.785326
Women_Women's Handbags_Totes & Shoppers          58.901964
Name: price, Length: 1239, dtype: float64

Predição dos preços de acordo com a categoria.

In [7]:
def baseline(categoria, medias):
    if categoria in medias.index:
        return medias.loc[categoria]
    else:
        return 3.0 # Caso a categoria não seja encontrada retorna o valor de 3 pois é o menor valor encontrado na base de dados
    

In [8]:
dados_teste['pred'] = dados_teste.apply(lambda x: baseline(x.categorias, medias_categorias), axis=1)

Tempo de execução = 16.2s

Avaliação do modelo considerando média geral.

In [11]:
def calc_metricas(obs, pred):
    r2 = metrics.r2_score(obs, pred).round(3)
    mape = (100 * metrics.mean_absolute_percentage_error(obs, pred)).round(3)
    mae = metrics.mean_absolute_error(obs, pred).round(2)
    rmse = (metrics.mean_squared_error(obs, pred)**0.5).round(2)
    msle = metrics.mean_squared_log_error(obs, pred).round(3)
    
    return [r2, mape, mae, rmse, msle]

def print_avaliacao(obs, pred):
    print('R² = %.3f' % metrics.r2_score(obs, pred))
    print('MAPE = %.3f %%' % (100 * metrics.mean_absolute_percentage_error(obs, pred)))
    print('MAE = U$S %.2f' % (metrics.mean_absolute_error(obs, pred)))
    print('RMSE = U$S %.2f' % metrics.mean_squared_error(obs, pred)**0.5)
    print('RMSLE = %.3f' % metrics.mean_squared_log_error(obs, pred, squared=False))

In [12]:
print_avaliacao(dados_teste['price'], dados_teste['pred'])

R² = 0.169
MAPE = 87.187 %
MAE = U$S 15.96
RMSE = U$S 35.29
MSLE = 0.699


Avaliação com amostra de validação

In [13]:
dados_validacao['pred'] = dados_validacao.apply(lambda x: baseline(x.categorias, medias_categorias), axis=1)

In [14]:
print_avaliacao(dados_validacao['price'], dados_validacao['pred'])

R² = 0.160
MAPE = 87.625 %
MAE = U$S 15.83
RMSE = U$S 35.14
MSLE = 0.700


Avaliação do modelo considerando a avaliação por categorias principais.

In [ ]:
avaliacoes = {}

for categoria in dados_teste.category_1.unique():
    dados_cat = dados_teste.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.101,0.220,0.152,0.270,0.028,0.151,0.109,0.097,0.169,0.191,-0.000
MAPE,88.604,79.545,111.558,116.893,76.874,74.482,91.109,77.969,85.241,81.851,109.691
MAE,17.160,19.830,19.190,25.150,10.650,11.110,12.640,13.240,14.610,10.590,16.620
RMSE,37.100,35.390,46.690,56.240,23.890,20.500,25.900,22.130,27.060,22.550,30.280
MSLE,0.504,0.454,0.648,0.667,0.416,0.398,0.503,0.431,0.472,0.441,0.641
